### Import Required Modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import pickle

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"

In [4]:
%cd ../../../
# %cd $PATH_TO_FOLDER

/home/souvic/mounted/btp/vahini/Name2Demographics


In [6]:
import sys  
sys.path.insert(0, 'Models/PreProcessing/')

from utils import *

sys.path.insert(0, 'Models/AIEEEData/Preprocessing/')
from aieee_caste import *

### Train Model on Data

In [7]:
caste_df = AIEEECasteData()

/home/souvic/mounted/btp/vahini/Name2Demographics/Models/AIEEEData/Preprocessing/aieee_caste.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col] = [elem.replace("Sub-PH", "") for elem in df1[col]]


In [8]:
train, val, test = splitTrainTestVal(caste_df, 0.7 ,0, 'Name')

(train:(178433, 4), val:(0, 0), test:(74986, 4))


In [9]:
total_points = caste_df.shape[0]
train_points = train.shape[0]
test_points = test.shape[0]
print(train_points/total_points)

0.7041026915898176


In [10]:
from sklearn.svm import SVC

In [11]:
svm_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 6), lowercase = True, stop_words = "english")),
    ('tfidf', TfidfTransformer(sublinear_tf = True)),
    ('clf', SVC(max_iter=10000, random_state=42))
])

In [12]:
svm_pipe.fit(train['Name'].values.astype('str'), train['Caste'])

/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 6), stop_words='english')),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf', SVC(max_iter=10000, random_state=42))])

In [14]:
y_pred=svm_pipe.predict(test['Name'].values.astype('str'))
print(classification_report(test['Caste'], y_pred, target_names = ['General', 'Reserved'], digits=4))

              precision    recall  f1-score   support

     General     0.6182    0.8895    0.7294     43478
    Reserved     0.6132    0.2418    0.3469     31508

    accuracy                         0.6173     74986
   macro avg     0.6157    0.5656    0.5381     74986
weighted avg     0.6161    0.6173    0.5687     74986



### Save Model

In [15]:
%pwd

'/home/souvic/mounted/btp/vahini/Name2Demographics'

In [16]:
filename = "Models/AIEEEData/SVM/SavedModel/AIEEEModel.pkl"
pickle.dump(svm_pipe, open(filename, 'wb'))

### Load Model

In [ ]:
# filename = "SavedModels/FinalSet/SGDClassifier/CBSEModel.pkl"
# pipe = pickle.load(open(filename, 'rb'))

In [ ]:
# df, first_name = split_name_df(cbse_df, 'Name')

In [ ]:
# y_pred=pipe.predict(df['first_name'].values.astype('str'))
# print(classification_report(test['Gender'], y_pred, target_names = ['Male', 'Female']))

In [ ]:
# fn = pd.DataFrame()

In [ ]:
# fn['first_name'] = df['first_name']

In [ ]:
# df['predict'] = y_pred

In [ ]:
# fn['gender'] = y_pred
# fn['true_gender'] = df['Gender']

In [ ]:
# print(fn.groupby(['first_name','gender']).size())

In [ ]:
# f1 = fn.drop_duplicates()
# f1 = f1.reset_index(drop=True)
# f1

,first_name,gender,true_gender
0,sarda,Girl,Girl
1,rakhi,Girl,Girl
2,surekha,Girl,Girl
3,shivani,Girl,Girl
4,ramdulari,Girl,Girl
...,...,...,...
25729,mannulal,Girl,Boy
25730,anada,Girl,Boy
25731,priyadarshan,Girl,Boy
25732,bindhyeshwari,Girl,Boy
